# TSA Chapter 3: ARIMA Forecasting with Expanding CI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch3/TSA_ch3_forecasting/TSA_ch3_forecasting.ipynb)

This notebook demonstrates:
- ARIMA forecasting: confidence intervals expand without bound (unlike ARMA)
- Demonstrates increasing uncertainty with horizon.


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
ar = np.array([1, -0.7])
ma = np.array([1, 0.4])
stationary = ArmaProcess(ar, ma).generate_sample(nsample=200)
data = np.cumsum(stationary)

model = ARIMA(data, order=(1, 1, 1)).fit()
h = 30
forecast = model.get_forecast(steps=h)
fc_mean = forecast.predicted_mean
fc_ci = forecast.conf_int(alpha=0.05)

if hasattr(fc_ci, 'iloc'):
    ci_lower, ci_upper = fc_ci.iloc[:, 0], fc_ci.iloc[:, 1]
else:
    ci_lower, ci_upper = fc_ci[:, 0], fc_ci[:, 1]

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
n = len(data)
ax.plot(range(n), data, color='#1A3A6E', linewidth=1.2, label='Observed')
fc_idx = range(n, n + h)
ax.plot(fc_idx, fc_mean, color='#DC3545', linewidth=2, linestyle='--', label='Forecast')
ax.fill_between(fc_idx, ci_lower, ci_upper, color='#DC3545', alpha=0.2, label='95% CI')
ax.axvline(x=n-1, color='gray', linestyle=':', alpha=0.5)
ax.set_xlim(n-50, n+h)
ax.set_title('ARIMA(1,1,1) Forecast: CI Expands Without Bound', fontweight='bold', fontsize=13)
ax.set_xlabel('Time')
ax.set_ylabel('Value')
ax.legend()

plt.tight_layout()
save_chart(fig, 'ch3_arima_forecast')
plt.show()

print("ARIMA vs ARMA Forecasting:")
print("  ARMA: CI converges to bounded width (stationary)")
print("  ARIMA: CI expands without bound (non-stationary)")
print("  Longer horizon = more uncertainty for ARIMA")